In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# Overview

In the following demo we illustrate how to compute the velocity $ \mathbf{v}(\mathbf{x},t) $ on a general three dimensional steady/unsteady velocity dataset. Particle trajectories $ \mathbf{x}(t;t_0,\mathbf{x_0}) $ launched from the initial position $ \mathbf{x}_0 $ at time $ t_0 $ satisfy the following ODE:

\begin{equation}
\mathbf{\dot{x}}(t) = \mathbf{v}(\mathbf{x}, t) \tag{1}
\end{equation}

The time-dependent velocity field $ \mathbf{v}(\mathbf{x}, t) $ is given on a gridded domain ($ X, Y, Z $). In this demo we choose the unsteady, viscous ABC flow. This notebook is structured as follows:

1. Import data from the file 'ABCunsteady.mat' stored in the folder 'data/ABC'.
2. Define computational parameters and data.
3. Interpolate velocity from (discrete) gridded data.
4. Compute velocity $ \mathbf{v}(\mathbf{x},t) $ at a point in space $ \mathbf{x} $ at time $ t $.

# Import data

In [2]:
# import scipy
import scipy.io as sio

# Import velocity data from file in data-folder
mat_file = sio.loadmat('../../data/ABC/ABCunsteady.mat')

U = mat_file['u'] # array (NY, NX, NZ)
V = mat_file['v'] # array (NY, NX, NZ)
W = mat_file['w'] # array (NY, NX, NZ)
x = mat_file['x'] # array (1, NX)
y = mat_file['y'] # array (1, NY)
z = mat_file['z'] # array (1, NZ)
time_data = mat_file['t']

# Computational parameters and data

Here we define the computational parameters and the data.

In [3]:
# import numpy
import numpy as np

# Number of cores for parallel computing
Ncores = 10 # int

# Periodic boundary conditions
periodic_x = True # bool
periodic_y = True # bool
periodic_z = True # bool
periodic = [periodic_x, periodic_y, periodic_z]

# Unsteady velocity field
bool_unsteady = True # bool

# Defined domain
defined_domain = np.isfinite(U).astype(int) # array (NY, NX, NZ)

## Compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z) # array (NY, NX), array (NY, NX, NZ)

## Resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0] # float
dy_data = Y[1,0,0]-Y[0,0,0] # float
dz_data = Y[0,0,1]-Z[0,0,0] # float

delta = [dx_data, dy_data, dx_data] # list (3, )

# Interpolate velocity

In order to evaluate the velocity field at arbitrary locations, we interpolate the discrete velocity data. The interpolation with respect to time and space is linear.

In [4]:
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0
W[np.isnan(W)] = 0

# Interpolate velocity data using linear spatial interpolation
Interpolant = interpolant_unsteady(X, Y, Z, U, V, W, time_data)

Interpolant_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_v = Interpolant[1] # RectangularBivariateSpline-object
Interpolant_w = Interpolant[2] # RectangularBivariateSpline-object

# IMPORTANT: We assume that the velocity field is on a gridded and evenly spaced domain. 
# We also assume for unsteady velocity fields, that the time is evenly spaced.

# Compute velocity field

In [9]:
# Import function to compute velocity
from ipynb.fs.defs.velocity import velocity

t = 3
#x = np.array([2, 1, 3]).reshape(3,-1)

x_domain = np.linspace(0, 1, 100)
y_domain = np.linspace(1, 2, 100)
z_domain = np.linspace(2, 3, 100)

X_domain, Y_domain, Z_domain = np.meshgrid(x_domain, y_domain, x_domain)

x = np.array([X_domain.ravel(), Y_domain.ravel(), Z_domain.ravel()]).reshape(3,-1)

vel = velocity(t, x, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)

(3, 1000000)


# References

[1] Chapter 2.2 Lagrangian Description of Fluid particles in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.

[2] [YouTube tutorial](https://youtu.be/VwfuLyv7cJw)